# Join con Pandas
Referencia: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

# Parte I

* Algunas de las operaciones más interesantes con los datos provienen de la combinación de diferentes fuentes de datos. Estas operaciones pueden ser

    1. simples concatenaciones de datos de datasets diferentes
    2. operaciones más similares a un join o merge en una base de datos

* Veremos dos operaciones: `pd.append` y `pd.concat`

In [2]:
import pandas as pd
import numpy as np

Vamos a crear algunos dataframes de ejemplo y trabajar sobre ellos.

In [3]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'C': ['C0', 'C1', 'C2', 'C3'],
                     'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])
 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                     'B': ['B4', 'B5', 'B6', 'B7'],
                     'C': ['C4', 'C5', 'C6', 'C7'],
                     'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])
 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                     'B': ['B8', 'B9', 'B10', 'B11'],
                     'C': ['C8', 'C9', 'C10', 'C11'],
                     'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])
 


## Concatenación simple con ``pd.concat``

* ``pd.concat()``, permite hacer concatenaciones simples de diferentes `Series`.

* Tiene una sintaxis similar a su análoga en Numpy ``np.concatenate``. Pero contiene algunas opciones más:

```python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

In [ ]:
frames = [df1, df2, df3]

result = pd.concat(frames)

* Por defecto, la concatenación se hace en el sentido de las filas del ``DataFrame`` (i.e., ``axis=0``).
* Puede especificarse el eje sobre cual realizar la concatenación:

### Índice duplicados

Recordemos que numpy también presenta un método de concatenación.

Una diferencia importante entre  ``np.concatenate`` y ``pd.concat`` es que la concatenación de Pandas preserva los índices, incluso si el resultado implica índices duplicados:

Si bien `DataFrame` permite índices duplicados es preferible eviatarlos.

#### Verificando la existencia índices duplicados

* Podemos verificar si existen índices solapados en el resultado de ``pd.concat()`` usando una ``verify_integrity`` flag.

* Al setear esto en True, la concatenación arrojará una excepción si existe algún índice duplicado:

In [4]:
# El parámetro verify_integrity evita que concatenemos dataframes con índices iguales.
pd.concat([df1, df2], verify_integrity=True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [5]:
# En general, para no recibir este comportamiento frente a un error, podemos generar una excepción para mostrarselo
# al usuario de forma más prolija
try:
    pd.concat([df1, df2], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

#### Ignorando el índice

* En algunos casos el índice no importa u podemos ignorarlo: simplemente usamos nuevamente ``ignore_index``. Al setearlo en True, se genera un nuevo índice.

### El método ``append()``

* Dado que la concatenación de arrays es bastante común, ``Series`` y ``DataFrame``tienen un método ``append`` 

* Por ejemplo, en lugar de llamar ``pd.concat([df1, df2])``, es posible llamar al más sencillo ``df1.append(df2)``:

In [6]:
result = df1.append(df2)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


* **IMPORTANTE:** tener en cuenta que, a diferencia del método `append` de listas, el `append()` de Pandas no modifica el objeto original. Genera un nuevo objeto con los datos combinados.
* Dado que esto implica crear un índice nuevo y un nuevo dataset `append` puede no ser el mejor método si se planea concatenar muchos datasets seguidos.
* En estos casos es mejor usar la función `pd.concat()`.

# PARTE II 

## Tipos de relaciones

Una de las características más valiosas de la librería Pandas es su funcionalidad para realizar joins en memoria y de manera eficiente.

El método merge() permite trabajar con objetos que presentan distintos tipos de relaciones:
 1. Uno a uno
 2. Muchos a uno
 3. Muchos a muchos
 

### 1. Join uno a uno

In [10]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                      'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']})
 

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                       'key2': ['K0', 'K0', 'K0', 'K0'],
                       'C': ['C0', 'C1', 'C2', 'C3'],
                       'D': ['D0', 'D1', 'D2', 'D3']})

In [11]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [12]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [13]:
result = pd.merge(left, right, on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


### 2. Join uno a muchos

### 3. Join muchos a muchos

## Joins por diferentes columnas

Puede suceder que en nuestros dataframes no haya una única columna con el mismo nombre en ambas tablas para poder hacer el join. 
Para resolver este problema, Pandas implementa los parámetros "on", "right_on" y "left_on" donde podemos especificar por cuáles columnas vamos a unir los datos. 

### 1. Join con "on"

### 2. Join con "left_on" y "right_on"

### 3. Join por más de una columna

## Tipos de joins

### 1. Left joins

### 2. Inner y outer join